## Using the MicMac command line scripts for Structure from Motion
----------------------

The following workflow demonstrates SfM on small dataset using some custom scripts in the **B**orne **A**gain **Sh**ell.

**Dependencies:**

An installation of MicMac and gdal.
    
The scripts site below with everything on your system path.

https://github.com/Ciaran1981/Sfm

You will need to install this python module to run bash commands without constant exclamation marks

```bash
pip install bash_kernel

python -m bash_kernel.install

```

QGIS 3 offers a versatile to visualise results so please ensure this is installed. Once installed, install the HCMGIS plugin to use various web-based base-layers for visualisation purposes. 

**Assumed knowledge:**

Basic GIS, basic image processing, remote sensing,and BASH use.

**The Data**

A subset of a larger dataset collected over an estuary in North Wales. The photos are accompanied by a csv file containing the GPS coordinates of each photo capture and a smaller subset for camera calibration. 

The data is temporarily found in this google drive folder until QinetiQ find somewhere to store it.

https://drive.google.com/open?id=1eM5pwMMwLSzp7S3fr-NJr6pzDi3tD0n3

The file required is **boatsorig.zip** - extract this somewhere appropriate and change dir to this folder.

This contains: 

- a collection of JPG photos
- boats.csv (GPS log)
- sub.csv (GPS subset for lens calibration)

**Lastlty**

If you wish to run every MicMac command a detailed account is given in the Sfm_guide.pdf in the Sfm repository. Otherwise, have a look at the content of the scripts themselves.


First we will change directory to where the photos are. The terminal will return the folder name so you can be sure!

In [1]:
cd /path/to/folder


### Orientation.sh
We will now run the first script which is named Orientation.sh and performs the feature detection, matching, orientation and GPS-aided bundle adjustment. The script has a few arguments - some mandatory others optional. 

- -e : image extenstion/file type (JPG, jpg,tif, png..., default=JPG).
- -u : UTM Zone of area of interest. Takes form 'NN +north(south)'
- -s : resize of imagery eg - 2000
- -c : Camera calibration model - e.g. RadialBasic, Fraser etc
- -m : Whether to manually mask the sparse cloud (longer processing!)
- -t : gps text file usually csv with mm3d formatting
- -s : a subset gps csv for pre-calibration of orientation

A typical usage is:

```bash
Orientation.sh -e JPG -u "30 +north" -c Fraser "30 +north" -t "calib.csv" -s "sub.csv" 
```
If an argument is omitted it is not used as an option within the script. The UTM zone (-u) and file arguments (-t, -s) must be in quotes!

This script (and every other) will present you with options as to whether you wish to continue based on the arguments you have given. This is to avoid frantically bashing ctrl-c to stop it if you forget something. 

The script can also be queried in the terminal as below.


In [2]:
Orientation.sh -h 1

Carry out feature extraction and orientation of images
Usage: Orientation.sh -e JPG -u -c Fraser 30 +north -s sub.csv 
	-e {EXTENSION}     : image file type (JPG, jpg, TIF, png..., default=JPG).
	-u UTMZONE       : UTM Zone of area of interest. Takes form 'NN +north(south)'
	-i SIZE         : resize of imagery eg - 2000
	-c CALIB        : Camera calibration model - e.g. RadialBasic, Fraser etc
	-m MASK        : Whether to manually mask the sparse cloud (longer processing!)
    -t CSV        : text file usually csv with mm3d formatting
    -s SUB        : a subset  csv for pre-calibration of orientation
	-h	             : displays this message and exits.
 


Run the script. perhaps best to go do something else at this point! A load of output nonsense will appear below but just ignore it for now. 

**You can omit the calibration subset as this not manadatory but illustrates the underlying process and may be a good option depending on time/dataset size constraints. Otherwize the MicMac Martini command is used to intialise the image block without radial distortion, which is then fed to the global orientation. This is internal to either python or bash-based methods.**

Simply remove the -s arg if you wish to run it automated

```bash
Orientation.sh -e JPG -u "30 +north" -c Fraser "30 +north" -t "calib.csv" 
```

In [ ]:
Orientation.sh -e JPG -u "30 +north" -c Fraser -t "boats.csv" -s "sub.csv" 

### dense_cloud.sh

For the final outputs, we will run the dense_cloud script, which as the name suggests, produces the final point cloud, DSM and ortho-mosaic.

The arguments are:

- -e : image extenstion/file type 
- -u : UTM Zone of area of interest. 
- -a : type of algorithm eg Ortho, UrbanMNE for Malt or MicMac, BigMac, QuickMac, Forest, Statue 
- -m : mode either Malt pyramidal MVS or PIMs (per-image-matching)
- -z : Last step in pyramidal dense correlation any of 1,2,3,4
- -i : use radiometric equalisation during orthomosaic stage
- -d : Degree of equalisation during orthomosaic stage
- -n : Number of cores to use 
- -r : zreg term - context dependent - reguralisation term   
- -o : do ortho -True or False  

```bash
dense_cloud.sh -e JPG -a Ortho -m Malt -u 30 +north -o true -i 1
```
We will omit some of these arguments as they are not usually required for good results. Of note here is that we specify -i as 1 to indicate radiometric equalisation to try and ensure even illumination in the mosaic. The ortho option is true by default.

The output files will be in a folder called OUTPUT (funnily enough), you will be able to open the DSM, orthomosaic and correlation images. The correlation image is representative of the normalised cross-correlation co-efficient for each pixel resulting from the processing. This can be interpeted as a proxy indication of how confident we are of each pixel height. The more consistently high values - the better. 

In [ ]:
dense_cloud.sh -e JPG -a Ortho -m Malt -u 30 +north -i 1

### Mosaicing alternatives

You may note that the mosaic output could be better from an illumination persepective. There are lots of seamlines between tiles (despite equalisation) and local contrast variation. Perhaps the one shortcoming of MicMac is that the orthomosaic function (Tawny) does not blend the edges of the individual ortho-images. It relies on equalising  illumination across the tiles and on occasion, despite various parameter choices may not cut it due to radiometric drift etc. Bear in mind, most of the time it is fine - this dataset is slightly problemtatic in this respect.

Whillst MicMac has an experimental seamline feathering command, it is at present slow to exectute and tends to deliver poorer results than the OSSIM version below. 

Likely the best open source implementation out there is that of the OSSIM library, but requires some script-based adaptation to work with MicMac outputs. It is the next script we will use. The script geo-refs all the individual orthos (the MicMac .tfw is insufficient for OSSIM to recognise the local projection), calculates their histograms then mosaics them.  

It is worth bearing in mind however, that by blending seamlines in certain areas, there is the potential for introducing artifacts in the imagery - there always a trade off in this respect (though it is barely noticable).  

The arguments for the script are:

-  -f : MicMac working directory or Malt/PIMs ortho dir 
-  -u : UTM Zone of area of interest
-  -m : whether to us maltbatch (bool) 
-  -p : whether to us pimsbatch (bool) 
-  -t : OSSIM mosaicing type  
-  -o : Output mosaic 

```bash 
orthomosaic.sh -f $PWD -u "30 +north" -t ossimFeatherMosaic -o outmosaic.tif

```
This will take a little while....

In [ ]:
orthomosaic.sh -f Ortho-MEC-Malt -u '30 +north' -t ossimFeatherMosaic -o OUTPUT/outmosaic.tif


### Some final notes

Here we have used the MicMac command line but as can be seen in the Sfm_guide.pdf, there are several software sources, which with tweaking will produce similar results - see Open Drone Map and Metashape as an example.